In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import itertools
import struct
import pandas as pd
from  tqdm import tqdm 
from tensorflow.keras.models import load_model
from keras.layers import Input, Dense, Reshape, Flatten,Dropout
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
DataPath="D:\\INSA_DATA\\Data\\LFP\\ResultTheo\\All\\"
Train_data=["cell1-25.h5","cell2-35.h5","cell3-45.h5"]
Test_data=["cell7-25.h5","cell8-35.h5","cell9-45.h5"]
maxI,minI,maxV,minV,maxT,minT=[9.381874,-20.00478,3.600254,1.989369,57.09709,18.68196]

In [3]:

def build_model(input_shape=(37500, 3), output_shape=25):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model



In [4]:
def Test(Steps_in,Steps_out):
    for stpsin,stpout in tqdm(zip(Steps_in,Steps_out)):
        data = np.load(f"Data\\Xtrain_normilized_{stpsin}_{stpout}.npz")
        X1 = data['Data']
        data = np.load(f"Data\\Ytrain_{stpsin}_{stpout}.npz")
        Y1 = data['y']
        data = np.load(f"Data\\Xtest_normilized_{stpsin}_{stpout}.npz")
        X2 = data['Data']
        data = np.load(f"Data\\Ytest_{stpsin}_{stpout}.npz")
        Y2 = data['y']
        X1 = np.float32(X1)
        Y1= np.float32(Y1)
        X2 = np.float32(X2)
        Y2= np.float32(Y2)

        model = build_model(input_shape=(1500*stpsin,3),output_shape=stpout)

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ModelCheckpoint(f"Models\\CNNLSTM_{stpsin}_{stpout}.keras", save_best_only=True)
        ]
        
        history = model.fit(
            X1, Y1,
            validation_split=0.1,
            epochs=1,# to be changed
            batch_size=32,
            callbacks=callbacks,
            verbose=0
        )

        model1=load_model(f"Models\\CNNLSTM_{stpsin}_{stpout}.keras")
        test_loss, test_mse = model1.evaluate(X2, Y2)
        print(f"\nTest MAE: {test_loss:.4f} | Test MSE: {test_mse:.4f}")

In [7]:
Steps_in=[25,25]
Steps_out=[100,200]    
Test(Steps_in,Steps_out)


0it [00:00, ?it/s]

213/213 ━━━━━━━━━━━━━━━━━━━━ 1001s 5s/step - loss: 0.1102 - mse: 0.0178


1it [3:43:43, 13423.31s/it]


Test MAE: 0.1002 | Test MSE: 0.0181
204/204 ━━━━━━━━━━━━━━━━━━━━ 786s 4s/step - loss: 0.0857 - mse: 0.0154


2it [9:16:13, 16686.94s/it]


Test MAE: 0.1025 | Test MSE: 0.0207


In [10]:
import pynvml
import time

Steps_in=[1,10,20,25,25,25,25]
Steps_out=[1,5,10,25,50,100,200] 
for stpsin,stpout in tqdm(zip(Steps_in,Steps_out)):
    data = np.load(f"Data\\Xtest_normilized_{stpsin}_{stpout}.npz")
    X2 = data['Data']
    model1=load_model(f"Models\\CNNLSTM_{stpsin}_{stpout}.keras")
    data = np.load(f"Data\\Ytest_{stpsin}_{stpout}.npz")
    Y2 = data['y']
    start=time.time()
    test_loss, test_mse= model1.evaluate(X2, Y2)
    end=time.time()
    test_rmse=np.sqrt(test_mse)
    test_time=(end-start)/len(X2)
    
    
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)

    start = time.time()
    power_start = pynvml.nvmlDeviceGetPowerUsage(handle)  # in mW

    # Run your inference
    output = model1(X2[0:1])

    power_end = pynvml.nvmlDeviceGetPowerUsage(handle)
    end = time.time()

    avg_power = (power_start + power_end) / 2  # in milliwatts
    duration = end - start  # in seconds

    energy_mJ = avg_power * duration  # mJ = mW * s
    print("MAE",test_loss,"MSE",test_mse,"RMSE",test_rmse,"RunningTime",duration,"Estimated energy",energy_mJ)
    results=[test_loss,test_mse,test_rmse,test_time,energy_mJ]
    np.save(f"Results\\resultsCNN_{stpsin}_{stpout}",results)

0it [00:00, ?it/s]

225/225 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - loss: 0.0887 - mse: 0.0158


1it [00:16, 16.34s/it]

MAE 0.10055549442768097 MSE 0.0202618595212698 RMSE 0.1423441587184729 RunningTime 1.3996152877807617 Estimated energy 28464.67591524124
224/224 ━━━━━━━━━━━━━━━━━━━━ 156s 695ms/step - loss: 0.0708 - mse: 0.0106


2it [03:11, 110.06s/it]

...

MAE 0.10042412579059601 MSE 0.019783474504947662 RMSE 0.14065373974746517 RunningTime 33.28957533836365 Estimated energy 576625.3792234659
213/213 ━━━━━━━━━━━━━━━━━━━━ 828s 4s/step - loss: 0.1102 - mse: 0.0178


6it [52:29, 701.40s/it]

MAE 0.10021448880434036 MSE 0.018054651096463203 RMSE 0.13436759689918995 RunningTime 33.74705266952515 Estimated energy 591602.7068231106
204/204 ━━━━━━━━━━━━━━━━━━━━ 798s 4s/step - loss: 0.0857 - mse: 0.0154


7it [1:06:36, 570.94s/it]

MAE 0.10251449048519135 MSE 0.02065172791481018 RMSE 0.14370709069078735 RunningTime 32.987467765808105 Estimated energy 570699.6860823631


In [16]:
def createModel(inputShape,output):
# define model
    model2 = Sequential()
    model2.add(LSTM(64, activation='relu',input_shape=inputShape,return_sequences=True ))
    model2.add(LSTM(32,activation='relu', return_sequences=False))
    model2.add(Dense(output,activation="sigmoid"))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model2.compile(optimizer=opt,loss='mae',metrics=['mae','mse',tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsolutePercentageError()])
    return model2

In [19]:
import pynvml
import time

In [24]:
def TestLSTM(Steps_in,Steps_out):
    for stpsin,stpout in tqdm(zip(Steps_in,Steps_out)):
        data = np.load(f"Data\\Xtrain_normilized_{stpsin}_{stpout}.npz")
        X1 = data['Data']
        data = np.load(f"Data\\Ytrain_{stpsin}_{stpout}.npz")
        Y1 = data['y']
        data = np.load(f"Data\\Xtest_normilized_{stpsin}_{stpout}.npz")
        X2 = data['Data']
        data = np.load(f"Data\\Ytest_{stpsin}_{stpout}.npz")
        Y2 = data['y']
        X1 = np.float32(X1)
        Y1= np.float32(Y1)
        X2 = np.float32(X2)
        Y2= np.float32(Y2)

        input_shape = (1500*stpsin,3)
        hidden_size1 = 256
        hidden_size2 = 128
        hidden_size3 = 100
        code_size = 64

        input_layer = Input(shape=(input_shape))

        hidden_layer_0=Flatten()(input_layer)
        hidden_layer_1 = Dense(hidden_size1, activation='relu')(hidden_layer_0)
        hidden_layer_2 = Dense(hidden_size2, activation='relu')(hidden_layer_1)
        hidden_layer_3 = Dense(hidden_size3, activation='relu')(hidden_layer_2)
        bottleneck_layer = Dense(code_size*stpsin, activation='sigmoid')(hidden_layer_3)

        # Define the decoder network
        hidden_layer_5 = Dense(hidden_size1, activation='relu')(bottleneck_layer)
        hidden_layer_6 = Dense(hidden_size2, activation='relu')(hidden_layer_5)
        hidden_layer_7 = Dense(hidden_size1, activation='relu')(hidden_layer_6)
        hidden_layer_8 = Dense(1500*stpsin*3, activation='sigmoid')(hidden_layer_7)
        output_layer=Reshape((1500*stpsin,3))(hidden_layer_8)

        autoencoder = Model(input_layer, output_layer)
        opt = Adam(learning_rate=0.0001)
        autoencoder.compile(optimizer=opt, loss='mae',metrics=['mae', 'mse',tf.keras.metrics.RootMeanSquaredError()])
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
        autoencoder.fit(X1, X1, epochs=300,batch_size=64,
                        validation_data=(X2, X2),callbacks=[early_stopping],verbose=0
                    )
        print("Encoder done")
        encoder = Model(input_layer, bottleneck_layer)
        encoder.compile(optimizer='adam', loss='mae')
        X11=encoder.predict(X1)
        X22=encoder.predict(X2)
        X11=np.float32(np.reshape(X11,(len(X11),stpsin,64)))
        X22=np.float32(np.reshape(X22,(len(X22),stpsin,64)))
        input_shape = (stpsin, 64)
        model2=createModel(input_shape,stpout)
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ModelCheckpoint(f"Models\\ELSTM_{stpsin}_{stpout}.keras", save_best_only=True)
        ]


        history = model2.fit(
            X11, Y1,
            validation_split=0.1,
            epochs=4,# to be changed
            batch_size=32,
            callbacks=callbacks,
            verbose=1)
        test_loss = model2.evaluate(X22, Y2)
        print("Model done")
        input_shape = (1500*stpsin,3)
        fsize=64

        input_layer = Input(shape=(input_shape))
        hidden_layer_0=encoder(input_layer)
        hidden_layer_1 =Reshape((stpsin,fsize))(hidden_layer_0)
        output_layer = model2(hidden_layer_1)

        model3 = Model(input_layer, output_layer)
        model3.compile(optimizer='adam', loss='mae')

        model3.save(f"Models\\ELSTMt_{stpsin}_{stpout}.keras")
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)

        start = time.time()
        power_start = pynvml.nvmlDeviceGetPowerUsage(handle)  # in mW

        # Run your inference
        output = model3(X1[0:1])

        power_end = pynvml.nvmlDeviceGetPowerUsage(handle)
        end = time.time()

        avg_power = (power_start + power_end) / 2  # in milliwatts
        duration = end - start  # in seconds

        energy_mJ = avg_power * duration  # mJ = mW * s
        print("MAE",test_loss[1],"MSE",test_loss[2],"RMSE",test_loss[3],"RunningTime",duration,"Estimated energy",energy_mJ)
        results=[test_loss[1],test_loss[2],test_loss[3],duration,energy_mJ]
        np.save(f"Results\\resultsLSTM_{stpsin}_{stpout}",results)

In [25]:
Steps_in=[1,10,20,25,25,25,25]
Steps_out=[1,5,10,25,50,100,200]    
TestLSTM(Steps_in,Steps_out)

0it [00:00, ?it/s]

Encoder done
237/237 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3293 - mae: 0.3293 - mean_absolute_percentage_error: 36.9428 - mse: 0.1279 - root_mean_squared_error: 0.3573 - val_loss: 0.0949 - val_mae: 0.0949 - val_mean_absolute_percentage_error: 15.1354 - val_mse: 0.0144 - val_root_mean_squared_error: 0.1198
Epoch 2/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1033 - mae: 0.1033 - mean_absolute_percentage_error: 14.4222 - mse: 0.0177 - root_mean_squared_error: 0.1330 - val_loss: 0.1924 - val_mae: 0.1924 - val_mean_absolute_percentage_error: 29.7814 - val_mse: 0.0456 - val_root_mean_squared_error: 0.2136
Epoch 3/4
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0739 - mae: 0.0739 - mean_absolute_percentage_error: 10.4742 - mse: 0.0104 - root_mean_squared_error: 0.1020 - val_loss: 0.2024 - val_mae: 0.2024 - val_mean_absolute_percentage_error: 30.9762 - val_mse: 0.0480 - val_root_mea

1it [01:28, 88.76s/it]

Encoder done
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Epoch 1/4
212/212 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2069 - mae: 0.2069 - mean_absolute_percentage_error: 24.9917 - mse: 0.0661 - root_mean_squared_error: 0.2521 - val_loss: 0.1672 - val_mae: 0.1672 - val_mean_absolute_percentage_error: 25.9487 - val_mse: 0.0349 - val_root_mean_squared_error: 0.1867
Epoch 2/4
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0312 - mae: 0.0312 - mean_absolute_percentage_error: 4.5415 - mse: 0.0025 - root_mean_squared_error: 0.0497 - val_loss: 0.0840 - val_mae: 0.0840 - val_mean_absolute_percentage_error: 13.2905 - val_mse: 0.0100 - val_root_mean_squared_error: 0.1000
Epoch 3/4
212/212 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0113 - mae: 0.0113 - mean_absolute_percentage_error: 1.5687 - mse: 3.2248e-04 - root_mean_squared_error: 0.0179 - val_loss: 0.0685 - val_mae: 0.0685 - val_mean_absolute_percentage_error: 10.9331 - val_mse: 0.0071 - val_root_mea

2it [15:44, 540.13s/it]

...

MAE 0.0437755361199379 MSE 0.004007094539701939 RMSE 0.06330161541700363 RunningTime 0.04900002479553223 Estimated energy 650.5243291854858
Encoder done
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
213/213 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Epoch 1/4
203/203 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.2817 - mae: 0.2817 - mean_absolute_percentage_error: 32.7170 - mse: 0.1146 - root_mean_squared_error: 0.3381 - val_loss: 0.1935 - val_mae: 0.1935 - val_mean_absolute_percentage_error: 27.7444 - val_mse: 0.0560 - val_root_mean_squared_error: 0.2366
Epoch 2/4
203/203 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0927 - mae: 0.0927 - mean_absolute_percentage_error: 11.3084 - mse: 0.0317 - root_mean_squared_error: 0.1765 - val_loss: 0.0608 - val_mae: 0.0608 - val_mean_absolute_percentage_error: 8.8132 - val_mse: 0.0086 - val_root_mean_squared_error: 0.0926
Epoch 3/4
203/203 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0293 - mae: 0.0293 - mean_absolute_percentage_error: 3.8580 - mse: 0.0069 - root_mean_s

6it [1:28:49, 1035.61s/it]

MAE 0.022316014394164085 MSE 0.0008811145671643317 RMSE 0.02968357317149639 RunningTime 0.08199882507324219 Estimated energy 1101.408218383789
Encoder done
216/216 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
204/204 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.2921 - mae: 0.2921 - mean_absolute_percentage_error: 33.8057 - mse: 0.1165 - root_mean_squared_error: 0.3407 - val_loss: 0.1414 - val_mae: 0.1414 - val_mean_absolute_percentage_error: 20.2177 - val_mse: 0.0393 - val_root_mean_squared_error: 0.1984
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0729 - mae: 0.0729 - mean_absolute_percentage_error: 9.0983 - mse: 0.0264 - root_mean_squared_error: 0.1613 - val_loss: 0.0706 - val_mae: 0.0706 - val_mean_absolute_percentage_error: 10.5245 - val_mse: 0.0092 - val_root_mean_squared_error: 0.0959
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0178 - mae: 0.0178 - mean_absolute_percentage_error: 2.3278 - mse: 0.0014 - root_me

7it [1:51:27, 955.33s/it] 

MAE 0.02291528880596161 MSE 0.0015535523416474462 RMSE 0.03941512852907181 RunningTime 0.05700111389160156 Estimated energy 755.6067657470703
